In [1]:
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

/opt/miniconda3/envs/jupyter/lib/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


2.8.0
2.8.3


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
SAMPLE = 1000000
EPOCHS = 20

# Florida voter
df = pd.read_csv('/opt/data/fl_voterreg/fl_reg_name_race_2022.csv.gz')
df.dropna(subset=['name_first', 'name_last'], inplace=True)
df['race'] = df.race.map({'native_indian': 'other', 'asian': 'asian', 'nh_black': 'nh_black', 'hispanic': 'hispanic', 'nh_white': 'nh_white', 'other': 'other', 'multi_racial': 'other', 'unknown': 'unknown'})
df

,name_last,name_first,race
0,Hessler-Smith,Jason,nh_white
1,Rogers,Renee,nh_white
2,Bartolome,Crystal,nh_white
3,Bailey,Donna,nh_white
4,Carlson,Greggory,nh_white
...,...,...,...
15455105,Ballew,Christina,nh_white
15455106,Watts,Mark,nh_white
15455107,McRae,Evelyn,nh_white
15455108,Ward,Stephanie,nh_white


In [3]:
sdf = df[df.race.isin(['unknown']) == False].groupby(['race']).sample(int(SAMPLE/5), random_state=21)
del df

# Additional features
sdf['name_first'] = sdf.name_first.str.title()
sdf['name_last'] = sdf.name_last.str.title()
sdf

,name_last,name_first,race
4180262,Bhikhari,Chandika,asian
2050170,Harilal,Sreekumar,asian
241801,Huynh,Lana,asian
5312038,Patel,Vaidehi,asian
13782727,Wang,Lingli,asian
...,...,...,...
11185683,Hinostroza,Jon,other
7373319,Holguin,Matthew,other
10664871,Butler,Russell,other
5528330,Jones,Daisha,other


In [4]:
rdf = sdf.groupby('race').agg({'name_last': 'count'})
rdf.to_csv('./fl_voter_reg/lstm/fl_name_five_cat_race_2022.csv', columns=[])
rdf

,name_last
race,
asian,200000
hispanic,200000
nh_black,200000
nh_white,200000
other,200000


In [5]:
sdf.groupby('race').agg({'name_last': 'nunique'})

,name_last
race,
asian,50546
hispanic,55285
nh_black,29799
nh_white,65731
other,68123


## Preprocessing the input data

In [6]:
# concat last name and first name
sdf['name_last_name_first'] = sdf['name_last'] + ' ' + sdf['name_first']

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_

# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = [w[1] for w in words]
num_words = len(words_list)
print("num_words = %d" % num_words)


def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

num_words = 1329
Max feature len = 46, Avg. feature len = 12


## Train a LSTM model

ref: http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [7]:
'''The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model

max_features = num_words # 20000
feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

800000 train sequences
200000 test sequences
Pad sequences (samples x time)
X_train shape: (800000, 20)
X_test shape: (200000, 20)
5 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (800000, 5)
y_test shape: (200000, 5)


In [8]:
print('Build model...')

model = Sequential()
model.add(Embedding(num_words, 32, input_length=feature_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Build model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 32)            42528     
                                                                 
 lstm (LSTM)                 (None, 128)               82432     
                                                                 
 dense (Dense)               (None, 5)                 645       
                                                                 
Total params: 125,605
Trainable params: 125,605
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=1)
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=1)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/20
22500/22500 [==============================] - 1130s 50ms/step - loss: 1.1266 - accuracy: 0.5452 - val_loss: 1.0652 - val_accuracy: 0.5737
Epoch 2/20
22500/22500 [==============================] - 973s 43ms/step - loss: 1.0558 - accuracy: 0.5795 - val_loss: 1.0306 - val_accuracy: 0.5908
Epoch 3/20
22500/22500 [==============================] - 922s 41ms/step - loss: 1.0298 - accuracy: 0.5923 - val_loss: 1.0122 - val_accuracy: 0.5985
Epoch 4/20
22500/22500 [==============================] - 849s 38ms/step - loss: 1.0153 - accuracy: 0.5984 - val_loss: 1.0009 - val_accuracy: 0.6029
Epoch 5/20
22500/22500 [==============================] - 810s 36ms/step - loss: 1.0060 - accuracy: 0.6032 - val_loss: 0.9955 - val_accuracy: 0.6072
Epoch 6/20
22500/22500 [==============================] - 894s 40ms/step - loss: 1.0000 - accuracy: 0.6061 - val_loss: 0.9905 - val_accuracy: 0.6100
Epoch 7/20
22500/22500 [==============================] - 923s 41ms/step - loss: 0.9959 - accura

In [10]:
print('Test score:', score)
print('Test accuracy:', acc)

Test score: 0.9723479151725769
Test accuracy: 0.6197550296783447


## Confusion Matrix

In [11]:
p = model.predict(X_test, verbose=2) # to predict probability
y_pred = np.argmax(p, axis=-1)
target_names = list(sdf.race.astype('category').cat.categories)
print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

6250/6250 - 38s - 38s/epoch - 6ms/step
              precision    recall  f1-score   support

       asian       0.75      0.67      0.71     40000
    hispanic       0.73      0.86      0.79     40000
    nh_black       0.59      0.73      0.65     40000
    nh_white       0.50      0.72      0.59     40000
       other       0.45      0.11      0.17     40000

    accuracy                           0.62    200000
   macro avg       0.60      0.62      0.58    200000
weighted avg       0.60      0.62      0.58    200000

[[26981  3447  2661  4641  2270]
 [  982 34593  1348  2392   685]
 [  795   870 29170  8293   872]
 [ 1025  2192  6538 28963  1282]
 [ 6343  6051  9938 13424  4244]]


## Save model

In [12]:
model.save('./fl_voter_reg/lstm/fl_all_fullname_lstm_5_cat_2022.h5')

In [13]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('./fl_voter_reg/lstm/fl_all_fullname_vocab_5_cat_2022.csv', index=False, encoding='utf-8')